# Code to run UE solver processes:

These instructions assume that a pre-step has been done to generate a networkx graph of the network.
Will also assume that we have converted the graph to a geojson and added appropriate extra fields (like city, county, etc.) If this is not the case - these steps can be completed in the **Converting networkx into geojson (only has to be done once)** cell below.

To run the solver, we require the following inputs:
* networkx graph
* TAZ demand
* TAZ shapefile


# Run solver
Below is the baseline run case:

In [7]:
from ue_solver import run_solver

taz_demand_f = 'resources/demand/TAZ_sf_demand.csv'
network_graph = 'resources/network/sf_secondary_Webster_Posey_cutlinks/network_graph.txt'
output_file = 'resources/results/sf_secondary_Webster_Posey_cutlinks/demand_1p5_gap100.txt'

run_solver.run_solver_full(network_graph, taz_demand_f, output_file, 
                           taz_shapefile='resources/shapefiles/bay_area_tazs',
                           demand_scale = 1.5, distance_factor=0, toll_factor=0, 
                           gap = 10.0)


started at 1486397881.31
Running FW algorithm
done in 3715.66877103 sec


### Converting networkx into geojson (only has to be done once)
If geojson exists for the network of interest, than should not need to run this cell.

In [17]:
from ue_solver.conversions import networkx_to_geojson
from ue_solver.modify_network import remove_duplicate_links
from ue_solver.modify_network import append_city_names

#inputs:
graph_f='resources/network/sf_secondary_Webster_Posey_cutlinks/network_graph.txt'
geojson_f ='resources/network/sf_secondary_Webster_Posey_cutlinks/network.geojson' 

networkx_to_geojson(graph_f, geojson_outf=geojson_f)

# if this is the first time creating a geojson from a networkx, probably want to 
# run the following clean-up:
remove_duplicate_links(geojson_inf=geojson_f, geojson_outf=geojson_f)

#to add city (and county) info:
ca_cities_shp = 'resources/shapefiles/ca_cities_2014'
append_city_names(ca_cities_shp, geojson_f)

<type 'str'>
77028
network already has city labels


# Modifying network: 
## 1. Update network capacity:

In [1]:
from ue_solver.modify_network import update_capacity

attr_dict = {'county': ['Alameda']}
# attr_dict = {'city': ['Berkeley', 'Oakland']}
percent_cap = .5
geojson_inf = 'resources/network/sf_secondary/network.geojson'
geojson_outf = 'resources/network/sf_secondary_alameda_0p5/network.geojson'
graph_outf = 'resources/network/sf_secondary_alameda_0p5/network_graph.txt'

update_capacity(attr_dict, percent_cap, geojson_inf, 
                geojson_outf, graph_outf)

## 2. Cut links:

In [4]:
from ue_solver.modify_network import cut_links

#Inputs:
geojson_inf = 'resources/network/sf_secondary/network.geojson'
links_f = 'resources/cut_links/Webster_Posey_cutlinks.csv'
geojson_outf = 'resources/network/sf_secondary_Webster_Posey_cutlinks/network.geojson'
graph_outf = 'resources/network/sf_secondary_Webster_Posey_cutlinks/network_graph.txt'

cut_links(geojson_inf, links_f, geojson_outf, graph_outf)

# Process results

## 1. Results to geojson

In [18]:
from ue_solver.process_results import results_to_geojson

#Inputs:
results_f = 'resources/results/sf_secondary_Webster_Posey_cutlinks/demand_1p5_gap100.txt'
geojson_inf = 'resources/network/sf_secondary_Webster_Posey_cutlinks/network.geojson'
geojson_outf = 'resources/results/sf_secondary_Webster_Posey_cutlinks/demand_1p5_gap100.geojson'

results_to_geojson(results_f, geojson_inf, geojson_outf)

## 2. VMT, VHT, Delay

In [19]:
from ue_solver.process_results import geoj_vmt_vht_delay
import time

start = time.time()
print 'started at', start

results_geoj = 'resources/results/sf_secondary_Webster_Posey_cutlinks/demand_1p5_gap100.geojson'
totalODflow = 1158029.0
min_speed = 0
cities_aggregate_output_file = 'resources/results/sf_secondary_Webster_Posey_cutlinks/vmt_vht_delay_cities.csv'
output_summary = 'resources/results/sf_secondary_Webster_Posey_cutlinks/vmt_vht_delay_summary.csv'
save_path = 'resources/results/sf_secondary_Webster_Posey_cutlinks/vmt_vht_delay.pdf'

geoj_vmt_vht_delay(results_geoj, cities_aggregate_output_file, output_summary, totalODflow, min_speed, save_path)

print 'done in', time.time()-start, 'sec'

started at 1486406163.51
12996836.0795 388126.612612 103352.715392
done in 36.9094541073 sec
